In [ ]:
from sympy.physics.units import *
from sympy import *

# Rounding:
import decimal
from decimal import Decimal as DX
def iso_round(obj, pv, rounding=decimal.ROUND_HALF_EVEN):
    import sympy
    """
    Rounding acc. to DIN EN ISO 80000-1:2013-08
    place value = Rundestellenwert
    """
    assert pv in set([
        # place value   #  round to:
        0.1,            #  1st digit after decimal
        0.01,           #  2nd
        0.001,          #  3rd
        0.0001,         #  4th
        0.00001,        #  5th
        0.000001,       #  6th
        0.0000001,      #  7th
        0.00000001,     #  8th
        0.000000001,    #  9th
        0.0000000001,   # 10th
        ])
    try:
        tmp = DX(str(float(obj)))
        obj = tmp.quantize(DX(str(pv)), rounding=rounding)
    except:
        for i in range(len(obj)):
            tmp = DX(str(float(obj[i])))
            obj[i] = tmp.quantize(DX(str(pv)), rounding=rounding)
    return obj

# LateX:
kwargs = {}
kwargs["mat_str"] = "bmatrix"
kwargs["mat_delim"] = ""
# kwargs["symbol_names"] = {FB: "F^{\mathsf B}", }

# Units:
(k, M, G ) = ( 10**3, 10**6, 10**9 )
(mm, cm, deg) = ( m/1000, m/100, pi/180)
Newton = kg*m/s**2
Pa     = Newton/m**2
MPa    = M*Pa
GPa    = G*Pa
kN     = k*Newton

half = S(1)/2

# ---

a, mbar = var("a, mbar")
x, y, z = var("x, y, z")
xii, eta, the = var("xi, eta, theta")

sub_list=[
    ( a,     half *m       ),
    ( mbar,    30 *kg/m**2 ),
    ]

pprint("\nJ of horiz. plate: xx-component / (m a⁴):")
tmp = integrate( y**2, (x, -a, a) )
tmp = integrate( tmp,  (y, -a, a) )
tmp *= mbar
Jxx_hori = tmp
tmp /= mbar *a**4
pprint(tmp)

pprint("\nJ of vert. plate: (xi, eta, theta)-components / (m a⁴):")
s2a = sqrt(2)*a
tmp = integrate( the**2, (xii, -s2a, s2a)     )
tmp = integrate( tmp,  (the, -half*a, half*a) )
tmp *= mbar
J11 = tmp
tmp = integrate( xii**2 + the**2, (xii, -s2a, s2a)     )
tmp = integrate( tmp,  (the, -a/4, a/4) )
tmp *= mbar
J22 = tmp
tmp = integrate( xii**2, (xii, -s2a, s2a)     )
tmp = integrate( tmp,  (the, -a/4, a/4) )
tmp *= mbar
J33 = tmp

J12 = 0
tmp = integrate( xii*the, (xii, -s2a, s2a)     )
tmp = integrate( tmp,  (the, -a/4, a/4) )
tmp *= - mbar
J13 = tmp
J23 = 0

J = Matrix([
    [J11, J12, J13],
    [J12, J22, J23],
    [J13, J23, J33]
    ])

tmp = J/ mbar/a**4
pprint(tmp)
pprint("\nJ of first vert. plate (phi = + 45°): (X,Y,Z)-components / (m a⁴):")
p = 45 *deg
c, s = cos(p), sin(p)
R = Matrix([
    [ c, s, 0],
    [-s, c, 0],
    [ 0, 0, 1]
    ])

Rt = R.transpose()
J1 = R*J*Rt
tmp = J1
tmp /= mbar*a**4
pprint(tmp)

pprint("\nJ of second vert. plate (phi = - 45°): (X,Y,Z)-components / (m a⁴):")
p = -45 *deg
c, s = cos(p), sin(p)
R = Matrix([
    [ c, s, 0],
    [-s, c, 0],
    [ 0, 0, 1]
    ])
Rt = R.transpose()
J2 = R*J*Rt
tmp = J2
tmp /= mbar*a**4
pprint(tmp)

pprint("\nJ of union of vert. plates: (X,Y,Z)-components / (m a⁴):")
Jv = J1 + J2
tmp = Jv
tmp /= mbar*a**4
pprint(tmp)

pprint("\nTotal Jxx / (kg m²):")
dx, dy, dz = 0, 0, a/4
area = 2 * (2*a*sqrt(2) * a/2)
mass = mbar*area
Jv[0,0] +=  dz*dz * mass
Jxx_vert = Jv[0,0]

tmp = Jxx_hori + Jxx_vert
tmp = tmp.subs(sub_list)
tmp = tmp.simplify()
tmp /= kg * m**2
tmp = iso_round(tmp,0.1)
pprint(tmp)


